In [1]:
import cv2
import numpy as np
from scipy.signal import butter, filtfilt, find_peaks


In [6]:
# Cargar clasificadores pre-entrenados
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# Cargar el video
cap = cv2.VideoCapture('Juan60.mp4')



In [3]:

fps = cap.get(cv2.CAP_PROP_FPS)
fps = round(fps)
print(fps)

60


In [7]:
def define_cheek_regions(eyes):
    top_y = min([eye[1] for eye in eyes])
    bottom_y = top_y + max([eye[3] for eye in eyes])
    left_x = min([eye[0] for eye in eyes])
    right_x = max([eye[0] + eye[2] for eye in eyes])
    
    cheek_width = (right_x - left_x) // 4
    cheek_height = (bottom_y - top_y) * 2
    
    left_cheek = [left_x - cheek_width, top_y, cheek_width, cheek_height]
    right_cheek = [right_x, top_y, cheek_width, cheek_height]
    
    return left_cheek, right_cheek



In [8]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) >= 2:  # Asegurando que se detecten al menos 2 ojos
            cheeks = define_cheek_regions(eyes)
            
            for cheek in cheeks:
                cx, cy, cw, ch = cheek
                cheek_roi = roi_color[cy:cy+ch, cx:cx+cw]
                
                # Aplicar filtro de piel y calcular área
                hsv = cv2.cvtColor(cheek_roi, cv2.COLOR_BGR2HSV)
                mask = cv2.inRange(hsv, (0, 10, 60), (20, 150, 255))
                skin_area = cv2.countNonZero(mask)
                
                # Visualización y análisis adicional si es necesario
                cv2.rectangle(roi_color, (cx, cy), (cx+cw, cy+ch), (0, 255, 0), 2)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    #cv2.imshow('Frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


: 

In [ ]:
color_signal = []



# Diseñar filtro de banda para la frecuencia cardíaca
nyq = 0.5 * fps
low = 0.5 / nyq
high = 4.0 / nyq
b, a = butter(1, [low, high], btype='band')

while True:
    # ... [Código previo para la detección de rostro y mejillas]
    
    for cheek in cheeks:
        cx, cy, cw, ch = cheek
        cheek_roi = roi_color[cy:cy+ch, cx:cx+cw]
        
        # Extraer la señal de color
        green_value = np.mean(cheek_roi[:, :, 1])  # Canal verde
        color_signal.append(green_value)
        
        # Si tenemos suficientes muestras, procesar la señal para obtener el pulso
        if len(color_signal) > fs * 5:  # Al menos 5 segundos de datos
            # Filtrar la señal
            filtered_signal = filtfilt(b, a, color_signal)
            
            # Encontrar picos
            peaks, _ = find_peaks(filtered_signal, distance=fs*0.6)
            
            # Calcular la frecuencia cardíaca
            if len(peaks) > 1:
                peak_distances = np.diff(peaks)
                mean_distance = np.mean(peak_distances)
                heart_rate = 60.0 / mean_distance
                print(f"Estimated Heart Rate: {heart_rate:.2f} BPM")
            
            # Limpiar la señal para evitar usar demasiada memoria
            color_signal = color_signal[int(fps):]
    
    # ... [Resto del código para visualización y control]

# ... [Liberar recursos]